Feature engineering

In [12]:
# Load merged 2009 data
df_2009 = pd.read_parquet("../data/merged/merged_2009.parquet")
df_2009.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 96 entries, 2009-05-31 to 2009-09-06
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   beach_id                96 non-null     float64
 1   beach_name              96 non-null     object 
 2   e_coli_cfu              96 non-null     float64
 3   water_quality_advisory  96 non-null     object 
 4   safety_status           96 non-null     object 
 5   station_name            96 non-null     object 
 6   precip_mm               96 non-null     float64
dtypes: float64(3), object(4)
memory usage: 6.0+ KB


In [16]:
import pandas as pd
import numpy as np

# Load merged 2009 data (datetime index assumed)
df_2009 = pd.read_parquet("../data/merged/merged_2009.parquet")

# --- Verify Structure ---
print("Current Index:", df_2009.index.name)  # Should be 'date' or similar
print("Columns:", df_2009.columns.tolist())

# --- Temporal Features ---
# 1. Weekend indicator
df_2009['is_weekend'] = df_2009.index.dayofweek.isin([5, 6]).astype(int)

# 2. Seasonal features
df_2009['day_of_year'] = df_2009.index.dayofyear
df_2009['sin_doy'] = np.sin(2 * np.pi * df_2009['day_of_year'] / 365)

# --- Precipitation Lags --- 
for lag in [1, 2, 3, 5, 7]:
    df_2009[f'precip_lag_{lag}d'] = df_2009['precip_mm'].shift(lag)

# --- Temperature Features ---
temp_cols = [c for c in df_2009.columns if 'temp' in c.lower()]  # Auto-detect
for temp_col in temp_cols[:3]:  # Use first 3 temp columns if many exist
    for lag in [1, 2]:
        df_2009[f'{temp_col}_lag{lag}d'] = df_2009[temp_col].shift(lag)

# --- E. coli Persistence ---
for lag in [1, 3, 7]:
    df_2009[f'e_coli_lag{lag}d'] = df_2009['e_coli_cfu'].shift(lag)

# --- Interaction Terms ---
if 'precip_lag_2d' in df_2009 and 'air_temp_avg_lag1d' in df_2009:
    df_2009['precip_x_temp'] = df_2009['precip_lag_2d'] * df_2009['air_temp_avg_lag1d']

# --- Drop NA ---
lag_cols = [c for c in df_2009.columns if 'lag' in c]
df_2009 = df_2009.dropna(subset=lag_cols)

print("Final columns:", df_2009.columns.tolist())

Current Index: date
Columns: ['beach_id', 'beach_name', 'e_coli_cfu', 'water_quality_advisory', 'safety_status', 'station_name', 'precip_mm']
Final columns: ['beach_id', 'beach_name', 'e_coli_cfu', 'water_quality_advisory', 'safety_status', 'station_name', 'precip_mm', 'is_weekend', 'day_of_year', 'sin_doy', 'precip_lag_1d', 'precip_lag_2d', 'precip_lag_3d', 'precip_lag_5d', 'precip_lag_7d', 'e_coli_lag1d', 'e_coli_lag3d', 'e_coli_lag7d']


Statsmodels regression

In [17]:
import statsmodels.api as sm

# Prepare data (drop non-feature columns and NAs)
regression_data = df_2009.drop(columns=[
    'beach_id', 'beach_name', 'water_quality_advisory', 
    'safety_status', 'station_name'
]).dropna()

# Define predictors (X) and target (y)
X = regression_data.drop(columns=['e_coli_cfu'])
y = regression_data['e_coli_cfu']

# Add constant for intercept term
X = sm.add_constant(X)

# Fit OLS regression
model = sm.OLS(y, X).fit()

# Print summary
print(model.summary())

# --- Diagnostic Checks ---
print("\n=== Key Insights ===")
print(f"1. R-squared: {model.rsquared:.3f} (Adj. {model.rsquared_adj:.3f})")
print(f"2. Most significant feature: {model.pvalues.idxmin()} (p={model.pvalues.min():.3g})")
print("3. Feature coefficients:")
print(model.params.sort_values(ascending=False))

                            OLS Regression Results                            
Dep. Variable:             e_coli_cfu   R-squared:                       0.323
Model:                            OLS   Adj. R-squared:                  0.216
Method:                 Least Squares   F-statistic:                     3.024
Date:                Sat, 05 Apr 2025   Prob (F-statistic):            0.00164
Time:                        15:04:26   Log-Likelihood:                -428.15
No. Observations:                  89   AIC:                             882.3
Df Residuals:                      76   BIC:                             914.6
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          -641.5271    332.257     -1.931